In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import cv2
from src.lab1.lab1_challenge2 import *
from src.lab1.info_image import *
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from src.lab3.lab3 import *
from tensorflow.train import Saver

SIZE = 28

### On extrait les données d'apprentissage

In [2]:
data = get_all_masks(100)
print(data[0][:10])
X_train = []; y_train = []
test_data = 0
# On crée les jeux d'entrainements....
for img_name, mask in data:
    img = cv2.imread(img_name)
    img2 = cv2.resize(img, None, fx=0.5, fy=0.5)
    mask2 = cv2.resize(mask, None, fx=0.5, fy=0.5)
    img3 = cv2.resize(img2, None, fx=0.5, fy=0.5)
    mask3 = cv2.resize(mask2, None, fx=0.5, fy=0.5)
    for (roi, center, w, h, angle) in sliding_windows(mask, SIZE, SIZE):
        x = extract_batch(img, center, SIZE, SIZE)
        y = extract_mask(mask, center, SIZE, SIZE)
        if x is not None and y is not None:
            X_train.append(x)
            y_train.append(y)
            test_data += 1

    for (roi, center, w, h, angle) in sliding_windows(mask2, SIZE, SIZE):
        x = extract_batch(img2, center, SIZE, SIZE)
        y = extract_mask(mask2, center, SIZE, SIZE)
        if x is not None and y is not None:
            X_train.append(x)
            y_train.append(y)
            test_data += 1
    for (roi, center, w, h, angle) in sliding_windows(mask3, SIZE, SIZE):
        x = extract_batch(img3, center, SIZE, SIZE)
        y = extract_mask(mask3, center, SIZE, SIZE)
        if x is not None and y is not None:
            X_train.append(x)
            y_train.append(y)
            test_data += 1
    
print("NUMBER OF TEST DATA", test_data)

['Images/2002/08/11/big/img_591.jpg', array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])]
NUMBER OF TEST DATA 73759


In [77]:
data = get_all_masks(20)
# Reads an image from a file, decodes it into a dense tensor, and resizes it
# to a fixed shape.
def _parse_function(filename, label):
    image_string = tf.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string)
    return image_decoded, label

filenames, labels = [], []
for name, mask in data:
    filenames.append(name)
    labels.append(tf.convert_to_tensor(mask))
    
dataset = tf.data.Dataset.from_tensor_slices((tf.constant(filenames), tf.constant(labels)))
dataset = dataset.map(_parse_function)


TypeError: List of Tensors when single Tensor expected

### On crée le modèle: On tente Vgg

In [84]:
input_size = (SIZE,SIZE,3)

model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=input_size, padding='same',
activation='relu'))
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Flatten())
#model.add(Dense(1024, activation='relu'))
model.add(Dense(2, activation="softmax"))

model.compile(optimizer=tf.train.AdamOptimizer(),
              loss='sparse_categorical_crossentropy',
              metrics=['mae', 'acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_236 (Conv2D)          (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_237 (Conv2D)          (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_92 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_238 (Conv2D)          (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_239 (Conv2D)          (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_93 (MaxPooling (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_240 (Conv2D)          (None, 8, 8, 256)         295168    
__________

In [5]:
input_size = (28,28,3)

model2 = Sequential()
model2.add(Conv2D(64, (3, 3), input_shape=input_size, padding='same',
activation='relu'))
model2.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model2.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model2.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model2.add(Flatten())
#model2.add(Dense(1024, activation='relu'))
model2.add(Dense(2, activation="softmax"))

model2.compile(optimizer=tf.train.AdamOptimizer(),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 28, 28, 64)        1792      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 14, 14, 128)       73856     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 14, 14, 128)       147584    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 128)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 6272)              0         
__________

### Entrainement du model Inchala 

In [6]:
X_train = np.array(X_train)

print(y_train[:10])
y_train = y_train

print("x train shape", X_train.shape)
print("y train shape", len(y_train))

history = model2.fit(X_train,
                    y_train, epochs = 5)


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
x train shape (73759, 28, 28, 3)
y train shape 73759
Epoch 1/5
73759/73759 [==============================] - 359s 5ms/step - loss: 0.2986 - acc: 0.8777
Epoch 2/5
73759/73759 [==============================] - 362s 5ms/step - loss: 0.2570 - acc: 0.8956
Epoch 3/5
73759/73759 [==============================] - 357s 5ms/step - loss: 0.2402 - acc: 0.9024
Epoch 4/5
73759/73759 [==============================] - 355s 5ms/step - loss: 0.2250 - acc: 0.9088
Epoch 5/5
73759/73759 [==============================] - 355s 5ms/step - loss: 0.2117 - acc: 0.9149


In [7]:
model2.save("./modele/perso_100_28_fenetre.h5")

### Test du modele


In [8]:
print(SIZE)
test_data = get_test_masks()[:3]
for img_name, mask in test_data:
    list_good = []
    list_to_predict = []
    img = cv2.imread(img_name)
    x_size = mask.shape[0]//SIZE
    y_size = mask.shape[1]//SIZE
    for (roi, center, w, h, angle) in sliding_windows(mask, SIZE, SIZE):
        d = extract_batch(img, center, SIZE, SIZE)
        m = extract_mask(mask, center, SIZE, SIZE)
        if d is not None and m is not None:
            list_good.append(m)
            list_to_predict.append(d)

    print("test batches", len(list_to_predict))
    prediction = model2.predict(np.array(list_to_predict))
    print("prediction shape", prediction.shape)
    print("image", img_name)

    img_reconstruct = np.zeros(img.shape)

    for x in range(x_size):
        for y in range(y_size):
            l = x * x_size + y
            print(prediction[l])
            if prediction[l][1] > 0.5:
                img_reconstruct[x*SIZE:(x+1)*SIZE, y*SIZE:(y+1)*SIZE] = img[x*SIZE:(x+1)*SIZE, y*SIZE:(y+1)*SIZE]

    print("saved image", img_name[0] + "test_model.png")
    cv2.imwrite(img_name[0] + "test_model.png", img_reconstruct)

28
test batches 575
prediction shape (575, 2)
image Images/2002/08/31/big/img_18008.jpg
[0.98525804 0.01474203]
[0.86114055 0.1388594 ]
[0.31608507 0.68391496]
[0.42253694 0.5774631 ]
[0.7245745 0.2754254]
[0.6187343  0.38126573]
[0.749436   0.25056407]
[0.95951074 0.04048931]
[9.9905044e-01 9.4956264e-04]
[9.999875e-01 1.256749e-05]
[9.9983966e-01 1.6027706e-04]
[9.9964786e-01 3.5214055e-04]
[0.99812025 0.00187978]
[0.99796826 0.00203173]
[0.99842143 0.00157859]
[0.99296135 0.00703862]
[0.99812025 0.00187978]
[0.99796826 0.00203173]
[0.99842143 0.00157859]
[0.99296135 0.00703862]
[0.68229717 0.3177029 ]
[0.8748151  0.12518495]
[0.98288673 0.01711322]
[0.99591666 0.00408336]
[0.99652123 0.00347878]
[0.9977366  0.00226338]
[0.99119544 0.0088045 ]
[0.47343707 0.5265629 ]
[0.19411403 0.805886  ]
[0.15033013 0.8496699 ]
[0.2212908  0.77870923]
[0.20121868 0.79878134]
[0.19411403 0.805886  ]
[0.15033013 0.8496699 ]
[0.2212908  0.77870923]
[0.20121868 0.79878134]
[0.07513016 0.92486984]
[0.3

test batches 440
prediction shape (440, 2)
image Images/2002/08/17/big/img_971.jpg
[0.9237855  0.07621443]
[0.8434859  0.15651408]
[0.8434989  0.15650119]
[0.993958   0.00604195]
[0.9379446  0.06205541]
[0.95092934 0.04907063]
[0.9793489  0.02065106]
[0.9534003 0.0465997]
[0.9555623  0.04443775]
[0.934406   0.06559405]
[0.9954379  0.00456208]
[0.97426414 0.02573586]
[0.9769711  0.02302895]
[0.97478074 0.02521929]
[0.96963185 0.03036815]
[0.9810983  0.01890172]
[0.8155225  0.18447746]
[0.75799716 0.24200287]
[0.78847224 0.21152778]
[0.9794631  0.02053682]
[0.98988605 0.01011398]
[0.983216   0.01678399]
[0.965859   0.03414103]
[0.96556026 0.03443969]
[0.9753221  0.02467793]
[0.9762122 0.0237878]
[0.9850958  0.01490419]
[0.9844318  0.01556827]
[0.9656691 0.0343309]
[0.9765276  0.02347247]
[0.9159687  0.08403129]
[9.9973148e-01 2.6851843e-04]
[0.8639434  0.13605657]
[0.96767825 0.03232178]
[0.9383361  0.06166394]
[0.9843843  0.01561564]
[0.9963342  0.00366585]
[0.97397095 0.02602906]
[0.92

In [ ]:
img = cv2.imread()